# 0. Instalación

In [5]:
%pip install scikit-learn pandas xgboost

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install imblearn SMOTE

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install transformers datasets torch scikit-learn huggingface_hub[hf_xet] transformers[torch]

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached torch-2.7.0-cp313-cp313-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached pyarrow-2

In [8]:
%pip install pandas numpy matplotlib seaborn scikit-learn

  Using cached matplotlib-3.10.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached contourpy-1.3.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.0-cp313-cp313-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (104 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached pillow-11.2.1-cp313-cp313-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached contourpy-1.3.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (322 

In [9]:
!pip install accelerate>=0.26.0

# 1. DistilBERT (fine-tuning)

**Descripción**

DistilBERT es una versión ligera y optimizada de BERT, un modelo de lenguaje basado en Transformers que permite realizar tareas de clasificación de texto con alta precisión. Este modelo se encarga de  captura las relaciones contextuales entre palabras en las frases, ofreciendo un rendimiento superior en tareas de procesamiento de lenguaje natural, como la clasificación de afirmaciones en verdadero o falso de fake news[1].

**Implementación**

En este proyecto, hemos trabajado con el dataset preprocesado y etiquetado. Para la tokenización y el entrenamiento hemos empleado el tokenizador y el modelo DistilBERT. El entrenamiento se realiza directamente sobre los datos preprocesados originales.

In [10]:
import pandas as pd
import datetime
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

/home/angel/LBBYs_CH2/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Cargamos el dataset de entrenamiento
data = pd.read_csv('../../../../data/processed/train_preprocess_v1.csv')

# Seleccionar las características
X = data['statement'] 
y = data['label'] 

Dividimos y tokenizamos con distilbert

In [12]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Cargar el tokenizador de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Realizamos la tokenización de los discursos del conjunto de entrenamiento y probamos usando un tokenizador de Hugging Face, asegurando que cada texto tenga una longitud máxima de 512 tokens y aplicando relleno para uniformizar su tamaño. Luego, convertimos los datos tokenizados y las etiquetas en un  Dataset para entrenar y evaluar el modelo.

In [13]:
# Tokenizar las afirmaciones de entrenamiento y prueba
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)

# Convertir a formato Dataset compatible con Hugging Face
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask'], 'label': y_train.tolist()})
test_dataset = Dataset.from_dict({'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask'], 'label': y_test.tolist()})


**Configuramos los parámetros**

Inicialmente, se realizó un entrenamiento de prueba con una configuración más rápida, reduciendo el número de épocas a 1 y disminuyendo el tamaño de los batches, tamaño reducido de statements que se procesa a la vez para mejorar el uso de memoria y sea más eficiente, tanto para entrenamiento como para evaluación. Esta configuración permitió validar rápidamente el pipeline y asegurarse de que el modelo iba a permitir proporcionar buenos resultados.

Sin embargo, aunque este entrenamiento rápido es útil para pruebas preliminares, resulta menos eficiente en términos de desempeño final del modelo. El número reducido de épocas limita la capacidad del modelo para aprender patrones complejos, y los batches pequeños pueden afectar la estabilidad y calidad del aprendizaje. Ya que 
- Batch pequeño: actualizaciones más frecuentes, pero es menos estable.

- Batch grande: actualizaciones menos frecuentes, pero más estables y con mejor estimación del gradiente, aunque requiere de más memoria y por lo tanto mayor tiempo computacional.

Por ello, en la configuración actual se incrementó el número de épocas a 3 y se utilizaron tamaños de batch mayores, permitiendo un entrenamiento más profundo y robusto. Esto mejora la capacidad del modelo para generalizar mejor a nuevos datos, aunque con un coste computacional elevado provocando tiempos de entrenamiento excesivamente largos, alcanzando las 4 horas de ejecuciación. Así, se busca un equilibrio óptimo entre precisión y recursos disponibles.

In [14]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Cargar el modelo DistilBERT para clasificación
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Configuración de los parámetros de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=8,   # batch de entrenamiento
    per_device_eval_batch_size=16,   # batch de evaluación
    warmup_steps=500,                # Número de pasos 
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,                
)

# Confiugración de modelo más básica
# training_args = TrainingArguments(
#     output_dir='./results',          
#     num_train_epochs=1,              # Usamos 1 época para prueba rápida
#     per_device_train_batch_size=4,   # batch de entrenamiento
#     per_device_eval_batch_size=8,    # batch de evaluación
# )

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Entrenamos el modelo

In [30]:
import numpy as np
from sklearn.metrics import f1_score
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)        
    labels = p.label_ids                           
    f1 = f1_score(labels, preds, average='macro')  
    return {'f1': f1}



# Entrenamiento del modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.708500
20,0.710100
30,0.716800
40,0.693400
50,0.687700
60,0.686200
70,0.661700
80,0.712300
90,0.657800
100,0.693000


c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=2352, training_loss=0.5511063802607206, metrics={'train_runtime': 37596.2394, 'train_samples_per_second': 0.5, 'train_steps_per_second': 0.063, 'total_flos': 2489724757739520.0, 'train_loss': 0.5511063802607206, 'epoch': 3.0})

Evaluamos el modelo

In [31]:
# Evaluar el modelo
trainer.evaluate()

# Predecir
pred_out = trainer.predict(test_dataset)
test_preds = np.argmax(pred_out.predictions, axis=1)

# Reporte completo de métricas
print(classification_report(y_test, test_preds))
print("Macro F1:", f1_score(y_test, test_preds, average='macro'))


c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


c:\Users\afgdl.ASUSS15ANGEL\Documents\GitHub\LBBYs_CH2\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


              precision    recall  f1-score   support

           0       0.49      0.45      0.47       923
           1       0.72      0.75      0.74      1762

    accuracy                           0.65      2685
   macro avg       0.61      0.60      0.60      2685
weighted avg       0.64      0.65      0.64      2685

Macro F1: 0.6032715196851095


## Conclusión

En resumen, DistilBERT demostró ser un modelo poderoso para tareas de clasificación de texto, destacándose por su eficiencia y efectividad, pero también dejando espacio para mejorar en términos de las métricas de evaluación. Se observa como se obtiene una accuracy de 0.65 y un Macro F1-Score de 0.60, lo que lo convierte en un modelo eficaz, aunque con espacio para mejoras. Este modelo ha demostrado un buen desempeño al identificar la clase mayoritaria (noticias verdaderas), alcanzando un recall del 75% y una precisión del 72%. Sin embargo, para la clase minoritaria (noticias falsas), el rendimiento fue más bajo, con un recall de 45% y una precisión de 49%, lo que refleja que el modelo tiene dificultades para detectar correctamente las noticias falsas, aunque aún proporciona predicciones aceptables.

En comparación con los otros modelos analizados, DistilBERT es el que obtiene el mejor rendimeinto, sin embargo, en ese análisis del modelo se observa como todavía existen ajustes y mejoras que permitan identificar con claridad la clase minoritaria.

# 2. Entrega Kaggle

## Distilbert

In [ ]:
import pandas as pd
import datetime
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from sklearn.metrics import classification_report, f1_score
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import Trainer, TrainingArguments, EvalPrediction
import numpy as np

# Cargar el dataset de test desde Kaggle (ruta del archivo que hayas cargado)
df_test = pd.read_csv('../../../../data/processed/test_preprocess_v1.csv')  # Asegúrate de usar la ruta correcta

# --- Preprocesamiento ---
# Cargar el tokenizador DistilBERT directamente desde Hugging Face
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenizar las afirmaciones de test
test_encodings = tokenizer(list(df_test['statement']), truncation=True, padding=True, max_length=512)

# Convertir a formato Dataset compatible con Hugging Face
test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'label': df_test['label'].tolist()  # Asume que df_test tiene la columna 'label'
})

# Cargar el modelo DistilBERT desde Hugging Face (sin necesidad de archivos previamente guardados)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir='./results',  # Carpeta para guardar los resultados
    num_train_epochs=1,      # Usamos 1 época para prueba rápida
    per_device_train_batch_size=4,  # Tamaño de batch reducido
    per_device_eval_batch_size=8,   # Tamaño de batch reducido
)

# Función de evaluación (cálculo de F1-score)
def compute_metrics(p: EvalPrediction):
    predictions, labels = p
    preds = np.argmax(predictions, axis=1)  # Convertir predicciones a etiquetas
    f1 = f1_score(labels, preds, average='macro')  # F1-score (macro)
    return {'f1': f1}

# Iniciar el entrenamiento con Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=test_dataset,  # Usar dataset de test como ejemplo para la predicción
    eval_dataset=test_dataset,   # Igual para evaluación
    compute_metrics=compute_metrics
)

# Entrenar el modelo (aunque solo sea por 1 época)
trainer.train()

# Evaluar el modelo (esto calculará las métricas de evaluación automáticamente)
trainer.evaluate()

# Realizar predicciones sobre test_dataset
pred_out = trainer.predict(test_dataset)

# Convertir las predicciones a etiquetas (tensores de PyTorch a numpy arrays)
test_preds = np.argmax(pred_out.predictions, axis=1)

# Reporte de métricas
print(classification_report(df_test['label'], test_preds))
print("Macro F1:", f1_score(df_test['label'], test_preds, average='macro'))

# --- Exportar Submission ---
# Crear el archivo CSV con las columnas requeridas: 'id' y 'label'
submission = pd.DataFrame({
    'id': df_test['id'],  # Usar el 'id' del dataset de prueba
    'label': test_preds  # Las predicciones del modelo
})

# Obtener la fecha actual en formato 'YYYY-MM-DD'
current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Generar el nombre del archivo con la fecha actual
filename = f'distilbert_{current_date}.csv'

# Guardar el archivo con el nombre que incluye la fecha
submission.to_csv(filename, index=False)

print(f"Archivo de submission '{filename}' generado correctamente.")


# 3. Referencias

1. [IA generativa, modelos ligeros y DistilBERT – Josué Cajahuamán Oscátegui](https://www.linkedin.com/pulse/ia-generativa-modelos-ligeros-distilbert-y-josu%C3%A9-cajahuam%C3%A1n-osc%C3%A1tegui-qxxge/)
